In [43]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### 1. Data loading and preprocessing

In [44]:
# Scaling the image (dividing a value of each pixels by 255, in order to get values from 0 to 1)  
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [48]:
# Loading the dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', shuffle_files=True, with_info=True, as_supervised=True)

# Dividing the dataset into train and test sets
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# Choosing the size for validation set (as 10% of the train set's size)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
print('num_validation_samples:', num_validation_samples)

# Saving the size of test set into variable
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
print('num_test_samples:', num_test_samples)

# Scaling train and validation data (dividing by 255)
scaled_train_and_validation_data = mnist_train.map(scale)

# Scaling test data (dividing by 255)
scaled_test_data = mnist_test.map(scale)

# Shuffling the data
buffer_size = 100

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

# Dividing into train and validation datasets
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Batching the train data
mini_batch_size = 100
train_data = train_data.batch(mini_batch_size)

# Batching validation and test data: they don't need to be batched, 
# but the model will require all data in batch form, so we take batch size as the whole set's size   
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

# make the same shape and object properties for validation data as for the train data (itarable)
validation_inputs, validation_targets = next(iter(validation_data))

num_validation_samples: tf.Tensor(6000, shape=(), dtype=int64)
num_test_samples: tf.Tensor(10000, shape=(), dtype=int64)


### 2. Model

In [49]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),                            
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

# Defining optimizer and loss function.
# sparse_categorical_crossentropy makes also a one-hot encoding for the data
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 3. Training the model

In [50]:
num_epochs = 8

model.fit(train_data, epochs=num_epochs, validation_data=(validation_inputs, validation_targets), validation_steps=1, verbose=2)

Epoch 1/8
540/540 - 8s - loss: 0.2681 - accuracy: 0.9188 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/8
540/540 - 8s - loss: 0.1061 - accuracy: 0.9674 - val_loss: 0.1554 - val_accuracy: 0.9582
Epoch 3/8
540/540 - 8s - loss: 0.0735 - accuracy: 0.9779 - val_loss: 0.1074 - val_accuracy: 0.9693
Epoch 4/8
540/540 - 8s - loss: 0.0574 - accuracy: 0.9815 - val_loss: 0.1113 - val_accuracy: 0.9698
Epoch 5/8
540/540 - 8s - loss: 0.0461 - accuracy: 0.9857 - val_loss: 0.1125 - val_accuracy: 0.9715
Epoch 6/8
540/540 - 8s - loss: 0.0363 - accuracy: 0.9889 - val_loss: 0.1322 - val_accuracy: 0.9692
Epoch 7/8
540/540 - 8s - loss: 0.0349 - accuracy: 0.9890 - val_loss: 0.1123 - val_accuracy: 0.9755
Epoch 8/8
540/540 - 8s - loss: 0.0292 - accuracy: 0.9916 - val_loss: 0.1223 - val_accuracy: 0.9733
